## Импорт библиотек

In [13]:
import pandas as pd
import sqlite3

## Соединение с базой данных

In [14]:
connection = sqlite3.connect('../data/checking-logs.sqlite')
connection.execute('DROP TABLE IF EXISTS datamart')

## Новая таблица datamart

In [15]:
query = 'CREATE TABLE IF NOT EXISTS datamart AS \
    SELECT ch.uid, ch.labname, ch.first_commit_ts, pg.first_view_ts \
        FROM (SELECT uid, labname, MIN(timestamp) AS first_commit_ts FROM checker \
            WHERE uid LIKE "user_%" AND labname IN ("laba04", "laba06s", "laba06", "laba04s", "laba05", "project1") AND status = "ready" AND numTrials = "1" GROUP BY labname, uid) AS ch \
                LEFT JOIN (SELECT uid, MIN(datetime) AS first_view_ts FROM pageviews WHERE uid LIKE "user_%" GROUP BY uid) AS pg \
                    ON ch.uid = pg.uid'
connection.execute(query)
datamart = pd.read_sql("SELECT * from datamart", connection)
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Тестовый и контрольный фреймы данных: 

In [16]:
test = datamart[datamart['first_view_ts'].notna()]
test.info()

control = datamart[datamart['first_view_ts'].isna()]
control.info()
control = control.copy()

test_mean = test['first_view_ts'].mean()
control['first_view_ts'] = control['first_view_ts'].fillna(test_mean)

test.to_sql('test', con=connection, if_exists='replace', index=False)
control.to_sql('control', con=connection, if_exists='replace', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 135
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 2 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


81

In [17]:
query = pd.read_sql("SELECT * FROM control", connection)
query

,uid,labname,first_commit_ts,first_view_ts
0,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783
1,user_13,laba04,2020-04-25 17:56:16.919886,2020-04-27 00:40:05.761783
2,user_15,laba04,2020-04-26 20:20:37.147886,2020-04-27 00:40:05.761783
3,user_16,laba04,2020-04-20 14:51:39.981342,2020-04-27 00:40:05.761783
4,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
...,...,...,...,...
76,user_29,project1,2020-05-11 17:57:37.768081,2020-04-27 00:40:05.761783
77,user_31,project1,2020-05-07 14:13:06.522558,2020-04-27 00:40:05.761783
78,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
79,user_6,project1,2020-05-13 19:50:34.262602,2020-04-27 00:40:05.761783


## Закрытие соединения с базой данных

In [18]:
connection.close()